In [9]:
mode='baseline_fb'
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from mne.decoding import CSP
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace,FGDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn import preprocessing
from sklearn.svm import SVC

import numpy as np
import moabb,sys,os,pickle
from moabb.datasets import BNCI2014001
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank
from pyriemann.estimation import Covariances
import pyriemann
from san_utils import *
from pyriemann.classification import MDM,FgMDM
### FB_MDM  ###


In [11]:
pipelines = {}
fb=FilterBank(make_pipeline(Covariances(estimator="oas")),flatten=False)

fb1 = FilterBank(make_pipeline(Covariances(estimator="oas"), CSP(nfilter=4)))
fb2 = FilterBank(make_pipeline(Covariances(estimator="oas"), TangentSpace()))
fb3 = FilterBank(make_pipeline(Covariances(estimator="oas"),FGDA(), TangentSpace()))

pipelines["FB_CSP_LDA"] = make_pipeline(fb1, LDA())
pipelines["FB_CSP_SVM"] = make_pipeline(fb1, SVC(kernel="linear"))
pipelines["FB_TS_LDA"] = make_pipeline(fb2, LDA())
pipelines["FB_TS_SVM"] = make_pipeline(fb2, SVC(kernel="linear"))
pipelines["FB_FGDA_TS_LDA"] = make_pipeline(fb3, LDA())
pipelines["FB_FGDA_TS_SVM"] = make_pipeline(fb3, SVC(kernel="linear"))
def save_pkl(xt,xe,yt,ye,le_map,direc):
    with open(direc+'/xt_cnn7.pkl', 'wb') as file:
        pickle.dump(xt, file)
    with open(direc+'/xe_cnn7.pkl', 'wb') as file:
        pickle.dump(xe, file)
    with open(direc+'/yt_cnn7.pkl', 'wb') as file:
        pickle.dump(yt, file)
    with open(direc+'/ye_cnn7.pkl', 'wb') as file:
        pickle.dump(ye, file) 
    with open(direc+'/le_map_cnn7.pkl', 'wb') as file:
        pickle.dump(le_map, file) 


In [13]:
dataset = BNCI2014001()
dataset_name="BNCI2014001"
subjects = [1, 2, 3,4,5,6,7,8,9]; n_class=4 ; sfreq=250 

allfilters=[[8, 12], [12, 16], [16, 20], [20, 24], [24, 28], [28, 34],[34,38]]

# allfilters=[[4,8],[8, 12], [12, 16], [16, 20], [20, 24], [24, 28], [28, 32],[32,36],[36,40],
#             [ 4., 10.],[10., 16.],[16., 22.],[22., 28.],[28., 34.],[34., 40.],
#            [ 4., 12.],[12., 20.],[20., 28.],[28., 36.],[8,30]]

# allfilters=[[8, 12], [12, 16], [16, 20], [20, 24], [24, 28],
#             [ 8, 14.],[14., 20.],[20., 26.],
#            [ 8, 16.],[16., 24.],[24., 32.],
#            [ 8., 10.],[10., 12.],[12., 14.],[14., 16.],[16., 18.],[18., 20.],[20., 22.],[22., 24.],[24., 26.]]
nfilt=len(allfilters)

paradigm = FilterBankMotorImagery(filters=allfilters,n_classes=n_class)
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc','recall','precision','f1','kappa'])
flg=0
xt={};yt={};xe={};ye={}
for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject])
    nchan=X.shape[1]
    if flg==0:
        le,le_label_map=label_map_fn(y) 
        flg=1
    select_indices = list(np.where(meta["session"] == "session_T")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_E")[0])
    X1=X[select_indices,:];y1=y[select_indices]
#     ind0 = [i for i, value in enumerate(y0) if ((value == value1) or (value ==value2))]
#     ind1 = [i for i, value in enumerate(y1) if ((value == value1) or (value ==value2))]
#     X_train=X0[ind0];  X_test=X1[ind1]; y_train =y0[ind0] ; y_test=y1[ind1]
#     n_class=2;le,le_label_map=label_map_fn(y_train)
    y_train = le.transform(y0);y_test = le.transform(y1)
    X_train=X0;X_test=X1
    ### save to cnn ###
    yt[(subject,len(allfilters))]=y_train; ye[(subject,len(allfilters))]=y_test; 
    fb.fit(X_train, ); xt[(subject,len(allfilters))]=fb.transform(X_train); xe[(subject,len(allfilters))]=fb.transform(X_test)
direc=dataset_name+'/'+mode
save_pkl(xt,xe,yt,ye,le_label_map,direc)
    ### save to cnn ###
    
#     #### FB CSP,TS classifiers ###
#     for pipeline in pipelines:
#         pipe=pipelines[pipeline]
#         pipe.fit(X_train, y_train)
#         y_pred=pipe.predict(X_test)
#         acc,recall,precision,f1,kappa=all_metrics(y_test,y_pred)
            
#         df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, pipeline, acc,recall,precision,f1,kappa]
#     #### FB_MDM ###
#     fb1 = FilterBank(make_pipeline(Covariances(estimator="oas"),MDM()), flatten=False)
#     a,y_pred=FB_MDM(X_train,y_train,X_test,y_test,fb1)
#     acc,recall,precision,f1,kappa=all_metrics(y_test,y_pred)
#     df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, "FB_MDM", acc,recall,precision,f1,kappa]
#     #### FB_FgMDM ###
#     fb2 = FilterBank(make_pipeline(Covariances(estimator="oas"),FgMDM()), flatten=False)
#     a,y_pred=FB_FgMDM(X_train,y_train,X_test,y_test,fb2) 
#     acc,recall,precision,f1,kappa=all_metrics(y_test,y_pred)
#     df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, nfilt, "FB_FgMDM", acc,recall,precision,f1,kappa]

Choosing from all possible events


subject: 1
subject: 2
subject: 3
subject: 4
subject: 5
subject: 6
subject: 7
subject: 8
subject: 9


In [ ]:
# saving the results
sfile_name = dataset_name+'/'+mode+'/Results_multiscale_fixed'+str(nfilt)+'bands.csv'

if(os.path.isfile('./'+sfile_name) ==True):
    print("exists")
    df.to_csv(sfile_name, mode="a", index=False, header=False)
else:
    print("No.. so creating")
    df.to_csv(sfile_name,  index=False, header=True)    


